In [1]:
%matplotlib inline

In [249]:
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [3]:
import tensorflow
tensorflow.keras.__version__

'2.4.0'

In [4]:
weather = pd.read_csv('WeatherDataPull/dataLargerSet.csv')
weather.head()

,Unnamed: 0,index,winery,vintage,points,cities,MarchLowTemp,MarchHighTemp,MarchAvgTemp,MarchPrecip,...,JulyAvgTemp,JulyPrecip,JulyCloudiness,JulyHumidity,AugLowTemp,AugHighTemp,AugAvgTemp,AugPrecip,AugCloudiness,AugHumidity
0,0,6,Claiborne & Churchill,2009,88,San Luis Obispo,42.129032,68.419355,58.580645,1.5,...,82.935484,0.0,2.612903,41.612903,56.741935,89.129032,79.903226,0.1,4.419355,44.000000
1,1,7,Claiborne & Churchill,2011,87,San Luis Obispo,43.806452,62.161290,55.258065,9.4,...,72.193548,0.1,1.838710,62.000000,61.741935,79.161290,73.806452,0.0,2.548387,57.000000
2,2,8,Claiborne & Churchill,2013,94,San Luis Obispo,47.354839,68.548387,60.516129,1.2,...,73.129032,0.2,5.322581,61.354839,61.290323,80.000000,73.806452,0.0,2.516129,59.806452
3,3,9,Claiborne & Churchill,2014,93,San Luis Obispo,48.612903,68.322581,61.129032,1.8,...,74.677419,0.3,10.032258,63.612903,62.096774,80.290323,73.967742,0.0,2.612903,63.774194
4,4,10,Claiborne & Churchill,2014,91,San Luis Obispo,48.612903,68.322581,61.129032,1.8,...,74.677419,0.3,10.032258,63.612903,62.096774,80.290323,73.967742,0.0,2.612903,63.774194


In [5]:
weatherClean=weather.drop(["Unnamed: 0", "index"], axis=1)
weatherClean.head()

,winery,vintage,points,cities,MarchLowTemp,MarchHighTemp,MarchAvgTemp,MarchPrecip,MarchCloudiness,MarchHumidity,...,JulyAvgTemp,JulyPrecip,JulyCloudiness,JulyHumidity,AugLowTemp,AugHighTemp,AugAvgTemp,AugPrecip,AugCloudiness,AugHumidity
0,Claiborne & Churchill,2009,88,San Luis Obispo,42.129032,68.419355,58.580645,1.5,12.870968,61.387097,...,82.935484,0.0,2.612903,41.612903,56.741935,89.129032,79.903226,0.1,4.419355,44.000000
1,Claiborne & Churchill,2011,87,San Luis Obispo,43.806452,62.161290,55.258065,9.4,35.032258,76.903226,...,72.193548,0.1,1.838710,62.000000,61.741935,79.161290,73.806452,0.0,2.548387,57.000000
2,Claiborne & Churchill,2013,94,San Luis Obispo,47.354839,68.548387,60.516129,1.2,21.387097,61.677419,...,73.129032,0.2,5.322581,61.354839,61.290323,80.000000,73.806452,0.0,2.516129,59.806452
3,Claiborne & Churchill,2014,93,San Luis Obispo,48.612903,68.322581,61.129032,1.8,22.290323,64.258065,...,74.677419,0.3,10.032258,63.612903,62.096774,80.290323,73.967742,0.0,2.612903,63.774194
4,Claiborne & Churchill,2014,91,San Luis Obispo,48.612903,68.322581,61.129032,1.8,22.290323,64.258065,...,74.677419,0.3,10.032258,63.612903,62.096774,80.290323,73.967742,0.0,2.612903,63.774194


In [251]:
weatherClean.sort_values(by='points')

,winery,vintage,points,cities,MarchLowTemp,MarchHighTemp,MarchAvgTemp,MarchPrecip,MarchCloudiness,MarchHumidity,...,JulyAvgTemp,JulyPrecip,JulyCloudiness,JulyHumidity,AugLowTemp,AugHighTemp,AugAvgTemp,AugPrecip,AugCloudiness,AugHumidity
97,Muscardini,2009,80,Kenwood,41.129032,62.774194,54.419355,4.9,28.838710,70.645161,...,78.967742,0.0,4.354839,47.677419,54.161290,88.161290,77.612903,0.0,6.354839,50.032258
261,Fenestra,2009,81,Livermore,42.096774,63.290323,55.290323,3.9,25.419355,74.322581,...,78.935484,0.0,4.000000,48.935484,56.000000,89.516129,78.548387,0.0,6.967742,51.451613
461,Peterson,2009,81,Healdsburg,41.548387,60.677419,53.193548,5.7,26.129032,70.935484,...,75.903226,0.0,2.580645,52.064516,55.451613,82.838710,73.935484,0.1,4.967742,54.612903
481,Kenwood,2011,81,Kenwood,36.645161,57.483871,50.096774,12.6,58.967742,86.354839,...,74.290323,0.0,5.806452,62.419355,50.387097,83.677419,73.161290,0.0,1.354839,59.774194
873,Adelaida,2009,81,Paso Robles,41.870968,67.451613,57.741935,1.1,16.064516,61.612903,...,85.806452,0.0,2.387097,32.516129,57.935484,92.161290,81.935484,0.1,4.516129,36.806452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,Gandona,2009,97,Saint Helena,41.129032,62.774194,54.419355,4.9,28.838710,70.645161,...,78.967742,0.0,4.354839,47.677419,54.161290,88.161290,77.612903,0.0,6.354839,50.032258
610,Williams Selyem,2010,98,Healdsburg,39.774194,60.322581,52.225806,4.5,29.129032,73.258065,...,74.064516,0.0,1.290323,56.387097,49.483871,84.870968,73.258065,0.0,3.096774,61.290323
626,Williams Selyem,2013,98,Healdsburg,40.225806,65.064516,55.838710,2.5,28.935484,70.032258,...,76.000000,0.0,2.967742,60.000000,53.161290,82.741935,72.645161,0.1,5.290323,63.870968
684,Alpha Omega,2012,99,Rutherford,35.870968,57.967742,50.032258,12.2,50.096774,78.129032,...,74.064516,0.0,5.774194,61.935484,52.645161,87.645161,75.516129,0.0,3.612903,55.967742


In [6]:
# Data Pre-Processing

In [237]:
weatherClean.columns

Index(['winery', 'vintage', 'points', 'cities', 'MarchLowTemp',
       'MarchHighTemp', 'MarchAvgTemp', 'MarchPrecip', 'MarchCloudiness',
       'MarchHumidity', 'AprilLowTemp', 'AprilHighTemp', 'AprilAvgTemp',
       'AprilPrecip', 'AprilCloudiness', 'AprilHumidity', 'MayLowTemp',
       'MayHighTemp', 'MayAvgTemp', 'MayPrecip', 'MayCloudiness',
       'MayHumidity', 'JuneLowTemp', 'JuneHighTemp', 'JuneAvgTemp',
       'JunePrecip', 'JuneCloudiness', 'JuneHumidity', 'JulyLowTemp',
       'JulyHighTemp', 'JulyAvgTemp', 'JulyPrecip', 'JulyCloudiness',
       'JulyHumidity', 'AugLowTemp', 'AugHighTemp', 'AugAvgTemp', 'AugPrecip',
       'AugCloudiness', 'AugHumidity'],
      dtype='object')

In [238]:
weatherCleandf = weatherClean.drop(['winery', 'vintage', 'cities'], axis=1)
weatherCleandf

,points,MarchLowTemp,MarchHighTemp,MarchAvgTemp,MarchPrecip,MarchCloudiness,MarchHumidity,AprilLowTemp,AprilHighTemp,AprilAvgTemp,...,JulyAvgTemp,JulyPrecip,JulyCloudiness,JulyHumidity,AugLowTemp,AugHighTemp,AugAvgTemp,AugPrecip,AugCloudiness,AugHumidity
0,88,42.129032,68.419355,58.580645,1.5,12.870968,61.387097,42.800000,72.166667,62.900000,...,82.935484,0.0,2.612903,41.612903,56.741935,89.129032,79.903226,0.1,4.419355,44.000000
1,87,43.806452,62.161290,55.258065,9.4,35.032258,76.903226,45.666667,68.500000,61.233333,...,72.193548,0.1,1.838710,62.000000,61.741935,79.161290,73.806452,0.0,2.548387,57.000000
2,94,47.354839,68.548387,60.516129,1.2,21.387097,61.677419,49.566667,71.566667,64.433333,...,73.129032,0.2,5.322581,61.354839,61.290323,80.000000,73.806452,0.0,2.516129,59.806452
3,93,48.612903,68.322581,61.129032,1.8,22.290323,64.258065,49.000000,71.066667,64.100000,...,74.677419,0.3,10.032258,63.612903,62.096774,80.290323,73.967742,0.0,2.612903,63.774194
4,91,48.612903,68.322581,61.129032,1.8,22.290323,64.258065,49.000000,71.066667,64.100000,...,74.677419,0.3,10.032258,63.612903,62.096774,80.290323,73.967742,0.0,2.612903,63.774194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,86,45.516129,60.129032,54.709677,2.8,16.129032,67.870968,49.233333,64.200000,59.266667,...,68.225806,0.3,3.387097,68.516129,63.129032,77.419355,72.806452,0.0,2.935484,57.451613
1324,92,51.064516,65.387097,59.806452,1.8,19.451613,62.741935,51.400000,67.433333,62.100000,...,72.838710,0.3,9.290323,64.806452,61.806452,77.193548,72.387097,0.0,1.870968,63.064516
1325,90,38.451613,62.064516,53.064516,4.0,32.645161,73.354839,39.366667,62.300000,54.733333,...,77.516129,0.0,2.000000,51.064516,47.806452,88.161290,74.838710,0.0,5.548387,57.677419
1326,89,35.870968,57.967742,50.032258,12.2,50.096774,78.129032,39.400000,66.633333,57.700000,...,74.064516,0.0,5.774194,61.935484,52.645161,87.645161,75.516129,0.0,3.612903,55.967742


In [239]:
# plt.scatter(weatherCleandf['points'],weatherCleandf['MarchPrecip'], color='red')
# plt.title('March Average Lows vs Wine Rating')
# plt.xlabel('weather')
# plt.ylabel('points')
# plt.show()

In [240]:
weatherCleanDF=weatherCleandf[['points','JuneCloudiness']]
weatherCleanDF

,points,JuneCloudiness
0,88,13.666667
1,87,11.000000
2,94,8.266667
3,93,1.233333
4,91,1.233333
...,...,...
1323,86,0.733333
1324,92,0.833333
1325,90,13.466667
1326,89,9.900000


In [241]:
X = weatherCleandf.drop(['points'], axis=1).values.astype(float)
y = weatherCleandf['points'].values.astype(float)
print(X)

[[42.12903226 68.41935484 58.58064516 ...  0.1         4.41935484
  44.        ]
 [43.80645161 62.16129032 55.25806452 ...  0.          2.5483871
  57.        ]
 [47.35483871 68.5483871  60.51612903 ...  0.          2.51612903
  59.80645161]
 ...
 [38.4516129  62.06451613 53.06451613 ...  0.          5.5483871
  57.67741935]
 [35.87096774 57.96774194 50.03225806 ...  0.          3.61290323
  55.96774194]
 [35.35483871 54.96774194 47.61290323 ...  0.          4.03225806
  58.90322581]]


In [242]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y.reshape(-1,1))

In [243]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [244]:
regressor = SVR(kernel='rbf')
regressor.fit(X,np.ravel(y,order="c"))

SVR()

In [245]:
y_pred = regressor.predict(X_test)
y_pred = sc_y.inverse_transform(y_pred)

In [246]:
df = pd.DataFrame({'Real Values':sc_y.inverse_transform(y_test.reshape(-1)), 'Predicted Values':y_pred})
df

,Real Values,Predicted Values
0,88.0,89.693227
1,86.0,87.874486
2,88.0,87.212258
3,92.0,90.218285
4,91.0,90.331341
...,...,...
261,91.0,89.014852
262,85.0,88.305683
263,88.0,88.251568
264,90.0,89.541679


In [247]:
training_score = regressor.score(X_train, y_train)
testing_score = regressor.score(X_test, y_test)

print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.18993451335364664
Testing Score: 0.22515257743146366
